# Final Project
## ExGender Bias 
__Lucas Calero Forero, Rebecca McBrayer__

In [51]:
import os
import string
import re

from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd

In [24]:
english_corpus = []
spanish_corpus = []
french_corpus = []
german_corpus = []

for filename in os.listdir('dev'):
    if '.en.' in filename:
        with open(f'dev/{filename}') as f:
            english_corpus.extend(f.readlines())
    if '.es.' in filename:
        with open(f'dev/{filename}') as f:
            spanish_corpus.extend(f.readlines())
    if '.fr.' in filename:
        with open(f'dev/{filename}') as f:
            french_corpus.extend(f.readlines())
    if '.de.' in filename:
        with open(f'dev/{filename}') as f:
            german_corpus.extend(f.readlines())

In [68]:
def preprocess(corpus):
    useful = []
    for item in corpus:
        if len(item)> 4 and '<seg' in item:
            start = item.find('">')
            end = item.find('</')
            useful.append(item[start+3:end].lower())

    # Remove punctuation
    exclude = set(string.punctuation)
    nopun = []
    for st in useful:
        st = ''.join(ch for ch in st if ch not in exclude)
        nopun.append(st)

    # Add start and ending tokens, and make all words lowercase
    data = [sentence.split() for sentence in nopun]
    data = list(filter(lambda a: len(a)>2, data))  # Remove blank sentences
    return data

In [70]:
english_data = preprocess(english_corpus)
spanish_data = preprocess(spanish_corpus)
french_data = preprocess(french_corpus)
german_data = preprocess(german_corpus)

In [99]:
def remove_articles(data, articles):
    clean_data = []
    for st in data:
        st = [word for word in st if not word in articles]
        clean_data.append(st)
    return clean_data

english_articles = ['the', 'a']
spanish_articles = ['el', 'uno', 'la', 'un', 'una', 'los', 'las', 'unos', 'unas']
french_articles = ['le', 'la', 'les', 'une', 'un']
german_articles = ['der', 'die', 'das', 'dem', 'den', 'ein', 'eine', 'einer', 'einem', 'einen', 'des']

english_no = remove_articles(english_data, english_articles)
spanish_no = remove_articles(spanish_data, spanish_articles)
french_no = remove_articles(french_data, french_articles)
german_no = remove_articles(german_data, german_articles)

In [72]:
english_model = Word2Vec(sentences=english_data, size=200, window=7, min_count=2, workers=4, sg=1)
spanish_model = Word2Vec(sentences=spanish_data, size=200, window=7, min_count=2, workers=4, sg=1)
french_model = Word2Vec(sentences=french_data, size=200, window=7, min_count=2, workers=4, sg=1)
german_model = Word2Vec(sentences=german_data, size=200, window=7, min_count=2, workers=4, sg=1)

In [102]:
english_model_no = Word2Vec(sentences=english_no, size=200, window=7, min_count=2, workers=4, sg=1)
spanish_model_no = Word2Vec(sentences=spanish_no, size=200, window=7, min_count=2, workers=4, sg=1)
french_model_no = Word2Vec(sentences=french_no, size=200, window=7, min_count=2, workers=4, sg=1)
german_model_no = Word2Vec(sentences=german_no, size=200, window=7, min_count=2, workers=4, sg=1)

In [73]:
print(f'English vocab size: {len(english_model.wv.vocab)}')
print(f'Spanish vocab size: {len(spanish_model.wv.vocab)}')
print(f'French vocab size: {len(french_model.wv.vocab)}')
print(f'German vocab size: {len(german_model.wv.vocab)}')

English vocab size: 17439
Spanish vocab size: 21096
French vocab size: 22095
German vocab size: 26759


In [103]:
print(f'English no article vocab size: {len(english_model_no.wv.vocab)}')
print(f'Spanish no article vocab size: {len(spanish_model_no.wv.vocab)}')
print(f'French no article vocab size: {len(french_model_no.wv.vocab)}')
print(f'German no article vocab size: {len(german_model_no.wv.vocab)}')

English no article vocab size: 17437
Spanish no article vocab size: 21087
French no article vocab size: 22090
German no article vocab size: 26749


In [108]:
print(english_model.wv.similarity('men', 'model'))
print(english_model.wv.similarity('women', 'model'))
print(english_model_no.wv.similarity('men', 'model'))
print(english_model_no.wv.similarity('women', 'model'))

0.6760016
0.86425006
0.63809365
0.8240618


In [109]:
print(spanish_model.wv.similarity('señor','modelo'))
print(spanish_model.wv.similarity('señora','modelo')) # masculine
print(spanish_model_no.wv.similarity('señor','modelo'))
print(spanish_model_no.wv.similarity('señora','modelo'))

0.95374393
0.83905363
0.9221438
0.83946764


In [110]:
print(french_model.wv.similarity('hommes', 'modèle'))
print(french_model.wv.similarity('femmes', 'modèle'))  # feminine
print(french_model_no.wv.similarity('hommes', 'modèle'))
print(french_model_no.wv.similarity('femmes', 'modèle'))

0.59201884
0.76704276
0.688988
0.81426114


In [111]:
print(german_model.wv.similarity('männer', 'model'))
print(german_model.wv.similarity('frauen', 'model')) # neutral
print(german_model_no.wv.similarity('männer', 'model'))
print(german_model_no.wv.similarity('frauen', 'model'))

0.7981342
0.7479851
0.86717266
0.83772045
